In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json
from io import StringIO

# Export

## Data

In [2]:
filepaths = []
subject = 1
session = 1
missing = 0
sub_total = 24 #24 is the default amount
events_master_df = pd.DataFrame(columns=['onset', 'duration', 'trial_type', 'response_time', 'sample', 'value','subject_id','session'])
channels_master_df = pd.DataFrame(columns=['name', 'type', 'units','subject_id','session'])

# Loop through 24 subject and their 2 sessions to concatenation all data into single datafram: master_df.
# List files not found and count total sessions concatenated.
while subject <=sub_total:
    if subject <10:
        sub = '00' + str(subject)
    else:
        sub = '0' + str(subject)
        
    while session <=2: #there are max 2 sessions per subject
        ses = '0' + str(session)
        events_filepath = f'../ds001787-1.0.2/sub-{sub}/ses-{ses}/eeg/sub-{sub}_ses-{ses}_task-meditation_events.tsv'
        channels_filepath = f'../ds001787-1.0.2/sub-{sub}/ses-{ses}/eeg/sub-{sub}_ses-{ses}_task-meditation_channels.tsv'

        try:
            events_df = pd.read_csv(events_filepath, sep='\t', header=0)
            channels_df = pd.read_csv(channels_filepath, sep='\t', header=0)
        except:
            print(events_filepath.split('/')[5],"does not exist.")
            print(channels_filepath.split('/')[5],"does not exist.") 
            missing += 1
            
#         print([f'sub-{sub}'] * len(events_df))
        events_df['subject_id'] = [f'sub-{sub}'] * len(events_df)
        events_df['session'] = [f'ses-{ses}'] * len(events_df)
        events_master_df = pd.concat([events_master_df,events_df])
        
        channels_df['subject_id'] = [f'sub-{sub}'] * len(channels_df)
        channels_df['session'] = [f'ses-{ses}'] * len(channels_df)
        channels_master_df = pd.concat([channels_master_df,channels_df])
    
        session+=1
        
    subject += 1
    session = 1
    
total_ses = sub_total*2 - missing
print(f"Finished concatenating {total_ses} sessions.")

sub-008_ses-02_task-meditation_events.tsv does not exist.
sub-008_ses-02_task-meditation_channels.tsv does not exist.
sub-012_ses-02_task-meditation_events.tsv does not exist.
sub-012_ses-02_task-meditation_channels.tsv does not exist.
sub-013_ses-02_task-meditation_events.tsv does not exist.
sub-013_ses-02_task-meditation_channels.tsv does not exist.
sub-014_ses-02_task-meditation_events.tsv does not exist.
sub-014_ses-02_task-meditation_channels.tsv does not exist.
sub-015_ses-02_task-meditation_events.tsv does not exist.
sub-015_ses-02_task-meditation_channels.tsv does not exist.
sub-019_ses-02_task-meditation_events.tsv does not exist.
sub-019_ses-02_task-meditation_channels.tsv does not exist.
sub-020_ses-02_task-meditation_events.tsv does not exist.
sub-020_ses-02_task-meditation_channels.tsv does not exist.
sub-021_ses-02_task-meditation_events.tsv does not exist.
sub-021_ses-02_task-meditation_channels.tsv does not exist.
sub-024_ses-02_task-meditation_events.tsv does not exist

In [3]:
participants_filepath = '../ds001787-1.0.2/participants.tsv'
ppts_df = pd.read_csv(participants_filepath, sep='\t', header=0)

# Transform

In [4]:
# Reorder columns and check if duration and response_time columns are entirely NaN. Drop if so.
events_master_df = events_master_df[['subject_id','session','onset','duration','trial_type','response_time','sample','value']]
if events_master_df.duration.isnull().all():
    events_master_df.drop(columns=['duration'])
if events_master_df.response_time.isnull().all():
    events_master_df.drop(columns=['response_time'])
events_master_df.astype({'subject_id':'object','session':'object','onset':'Float64','duration':'Float64','trial_type':'object','response_time':'Float64','sample':'Int64','value':'Int64'}).dtypes
events_master_df.set_index(['subject_id','session','onset'],inplace=True)
events_master_df.head(10)

duration trial_type  response_time   sample  \
subject_id session onset                                                     
sub-001    ses-01  71.386719        NaN   stimulus            NaN  18275.0   
                   75.730469        NaN   response            NaN  19387.0   
                   79.773438        NaN   response            NaN  20422.0   
                   125.609375       NaN   stimulus            NaN  32156.0   
                   179.800781       NaN   stimulus            NaN  46029.0   
                   183.097656       NaN   response            NaN  46873.0   
                   185.632812       NaN   response            NaN  47522.0   
                   284.859375       NaN   stimulus            NaN  72924.0   
                   287.757812       NaN   response            NaN  73666.0   
                   289.960938       NaN   response            NaN  74230.0   

                              value  
subject_id session onset             
sub-001    ses-01  71.386719    128  
                   75.730469      2  
                   79.773438      2  
                   125.609375   128  
                   179.800781   128  
                   183.097656     2  
                   185.632812     4  
                   284.859375   128  
                   287.757812     2  
                   289.960938     2

In [5]:
channels_master_df = channels_master_df[['subject_id', 'session','name', 'type', 'units']]
channels_master_df.astype('object').dtypes
channels_master_df.set_index(['subject_id','session','name'],inplace=True)
channels_master_df.head()

type   units
subject_id session name             
sub-001    ses-01  A1    EEG  microV
                   A2    EEG  microV
                   A3    EEG  microV
                   A4    EEG  microV
                   A5    EEG  microV

In [6]:
ppts_df.astype({'participant_id':'object','gender':'object','age':'Int64','group':'object'})
ppts_df.rename(columns={'participant_id':'subject_id'}, inplace=True)
ppts_df.set_index('subject_id',inplace=True)
ppts_df

,gender,age,group
subject_id,,,
sub-001,M,32.0,expert
sub-002,M,35.0,expert
sub-003,F,41.0,expert
sub-004,M,29.0,expert
sub-005,F,34.0,expert
sub-006,M,32.0,expert
sub-007,M,32.0,expert
sub-008,M,32.0,expert
sub-009,M,43.0,expert


# Load

### Create database connection

In [7]:
connection_string = "postgres:Ch3vr!3r17@localhost:5432/eeg_db"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Confirm tables
engine.table_names()

['eeg_events', 'eeg_channels', 'participants']

### Load Database into PostGres

In [9]:
events_master_df.to_sql(name='eeg_events', con=engine, if_exists='replace', index=True)
channels_master_df.to_sql(name='eeg_channels', con=engine, if_exists='replace', index=True)
ppts_df.to_sql(name='participants', con=engine, if_exists='replace', index=True)

In [10]:
# Confirm tables
# engine.table_names()
pd.read_sql_query('select * from eeg_events',con=engine).head()

,subject_id,session,onset,duration,trial_type,response_time,sample,value
0,sub-001,ses-01,71.386719,None,stimulus,None,18275.0,128
1,sub-001,ses-01,75.730469,None,response,None,19387.0,2
2,sub-001,ses-01,79.773438,None,response,None,20422.0,2
3,sub-001,ses-01,125.609375,None,stimulus,None,32156.0,128
4,sub-001,ses-01,179.800781,None,stimulus,None,46029.0,128


In [11]:
pd.read_sql_query('select * from eeg_channels',con=engine).head()

,subject_id,session,name,type,units
0,sub-001,ses-01,A1,EEG,microV
1,sub-001,ses-01,A2,EEG,microV
2,sub-001,ses-01,A3,EEG,microV
3,sub-001,ses-01,A4,EEG,microV
4,sub-001,ses-01,A5,EEG,microV


In [12]:
pd.read_sql_query('select * from participants',con=engine).head()

,subject_id,gender,age,group
0,sub-001,M,32.0,expert
1,sub-002,M,35.0,expert
2,sub-003,F,41.0,expert
3,sub-004,M,29.0,expert
4,sub-005,F,34.0,expert
